In [1]:
spark

Waiting for a Spark session to start...

Spark application_1533622723243_21712: Some(http://gw02.itversity.com:4051)

In [2]:
spark.sparkContext.getConf.getAll.foreach(x=>if(x._2.contains("/proxy/")){println(x._2)})

Waiting for a Spark session to start...

http://rm01.itversity.com:19288/proxy/application_1533622723243_21712


In [3]:
import scala.collection.JavaConversions._

In [4]:
val os_name = System.getProperty("os.name")
val hdfs_home = "/user/" + System.getProperty("user.home").split("/")(2)

os_name = Linux
hdfs_home = /user/kranthidr


/user/kranthidr

In [5]:
val path = hdfs_home+"/dataSets/wikidump-02-sept-2018.xml"

path = /user/kranthidr/dataSets/wikidump-02-sept-2018.xml


/user/kranthidr/dataSets/wikidump-02-sept-2018.xml

In [6]:
import com.cloudera.datascience.lsa._
import com.cloudera.datascience.lsa.AssembleDocumentTermMatrix
import com.cloudera.datascience.lsa.LSAQueryEngine
import com.cloudera.datascience.lsa.RunLSA._

In [7]:
import breeze.linalg.{DenseMatrix => BDenseMatrix, SparseVector => BSparseVector}

import org.apache.spark.mllib.linalg.{Matrices, Matrix, SingularValueDecomposition, Vectors, Vector => MLLibVector}
import org.apache.spark.ml.linalg.{Vector => MLVector}
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import org.apache.spark.serializer.KryoSerializer
import org.apache.spark.sql.{Dataset, SparkSession}

import scala.collection.Map
import scala.collection.mutable.ArrayBuffer

import breeze.linalg.{DenseMatrix=>BDenseMatrix, SparseVector=>BSparseVector}
import org.apache.spark.mllib.linalg.{Matrices, Matrix, SingularValueDecomposition, Vectors, Vector=>MLLibVector}
import org.apache.spark.ml.linalg.{Vector=>MLVector}


In [8]:
val k =  100
val numTerms = 20000

k = 100
numTerms = 20000


20000

In [9]:
val assembleMatrix = new AssembleDocumentTermMatrix(spark)
import assembleMatrix._

assembleMatrix = com.cloudera.datascience.lsa.AssembleDocumentTermMatrix@36dfb57f


com.cloudera.datascience.lsa.AssembleDocumentTermMatrix@36dfb57f

In [10]:
val docTexts: Dataset[(String, String)] = parseWikipediaDump(path)

docTexts = [_1: string, _2: string]


[_1: string, _2: string]

In [13]:
docTexts.cache()

lastException: Throwable = null


[_1: string, _2: string]

In [14]:
val docTexts1 = docTexts.sample(false, 0.001) //50 Docs we get approx

docTexts1 = [_1: string, _2: string]


[_1: string, _2: string]

In [15]:
docTexts1.count()

5656

In [16]:
val (docTermMatrix, termIds, docIds, termIdfs) = documentTermMatrix(docTexts1, "stopwords.txt", numTerms)

docTermMatrix = [title: string, tfidfVec: vector]
termIds = Array(also, new, first, one, year, reference, use, two, include, school, time, make, may, name, team, state, university, work, station, play, world, link, become, class, city, win, see, national, external, film, defaultsort, united, bear, take, american, part, release, john, season, cross, group, member, can, later, family, south, know, three, list, circle, march, award, area, people, game, october, second, war, end, september, follow, album, district, november, many, january, high, lead, county, place, road, april, club, call, song, company, history, august, december, show, states, college, write, june, begin, league, note, july, number, house, music, record, north, series, day, e...


[also, new, first, one, year, reference, use, two, include, school, time, make, may, name, team, state, university, work, station, play, world, link, become, class, city, win, see, national, external, film, defaultsort, united, bear, take, american, part, release, john, season, cross, group, member, can, later, family, south, know, three, list, circle, march, award, area, people, game, october, second, war, end, september, follow, album, district, november, many, january, high, lead, county, place, road, april, club, call, song, company, history, august, december, show, states, college, write, june, begin, league, note, july, number, house, music, record, north, series, day, early, york, since, town, february, form, life, church, man, give, find, center, international, party, well, service, west, market, leave, serve, result, the, move, hold, park, title, british, will, base, start, live, however, best, general, die, feature, open, hospital, force, nagar, book, point, art, come, four, 

In [17]:
docTexts.unpersist()

[_1: string, _2: string]

In [18]:
docTermMatrix.cache()

[title: string, tfidfVec: vector]

In [19]:
 val vecRdd = docTermMatrix.select("tfidfVec").rdd.map { row =>
      Vectors.fromML(row.getAs[MLVector]("tfidfVec"))
}

vecRdd = MapPartitionsRDD[83] at map at <console>:53


MapPartitionsRDD[83] at map at <console>:53

In [20]:
docTermMatrix.unpersist()

[title: string, tfidfVec: vector]

In [21]:
vecRdd.cache()

MapPartitionsRDD[83] at map at <console>:53

In [22]:
    val mat = new RowMatrix(vecRdd)

mat = org.apache.spark.mllib.linalg.distributed.RowMatrix@1346ee39


org.apache.spark.mllib.linalg.distributed.RowMatrix@1346ee39

In [23]:
    val svd = mat.computeSVD(k, computeU=true)

svd = 


SingularValueDecomposition(org.apache.spark.mllib.linalg.distributed.RowMatrix@632bdee3,[20964.681174349887,8365.1409827682,7678.370135785315,3995.9475058426497,2604.1153683332022,2215.6113875982765,2159.1507354762653,2135.542853011511,2076.7848133947114,1994.8757245892573,1908.3033366655288,1870.2822572100592,1784.2317969516248,1710.0881291070773,1682.8141263421649,1608.5146194121905,1596.9894418888152,1504.5684818114628,1415.5735080386517,1365.4349577363,1333.430901783591,1276.166374742556,1250.7505040186825,1237.2148153820065,1232.9389359691145,1227.8328438730168,1218.516185371996,1185.2920058110433,1174.5800060404586,1172.069614492...


SingularValueDecomposition(org.apache.spark.mllib.linalg.distributed.RowMatrix@632bdee3,[20964.681174349887,8365.1409827682,7678.370135785315,3995.9475058426497,2604.1153683332022,2215.6113875982765,2159.1507354762653,2135.542853011511,2076.7848133947114,1994.8757245892573,1908.3033366655288,1870.2822572100592,1784.2317969516248,1710.0881291070773,1682.8141263421649,1608.5146194121905,1596.9894418888152,1504.5684818114628,1415.5735080386517,1365.4349577363,1333.430901783591,1276.166374742556,1250.7505040186825,1237.2148153820065,1232.9389359691145,1227.8328438730168,1218.516185371996,1185.2920058110433,1174.5800060404586,1172.0696144921542,1126.0950334256443,1098.4033437840824,1089.7031091561817,1088.2444238609328,1074.3003087780903,1051.9865152827506,1045.538404935172,1031.9489812045092,1019.5008345827982,1007.3733174449126,1004.3706705820103,996.346152150619,979.2985872746425,967.3986160126037,964.797357117872,942.2002520007163,934.1737200866155,927.0953746055075,917.402819899194,916

In [24]:
    println("Singular values: " + svd.s)

Singular values: [20964.681174349887,8365.1409827682,7678.370135785315,3995.9475058426497,2604.1153683332022,2215.6113875982765,2159.1507354762653,2135.542853011511,2076.7848133947114,1994.8757245892573,1908.3033366655288,1870.2822572100592,1784.2317969516248,1710.0881291070773,1682.8141263421649,1608.5146194121905,1596.9894418888152,1504.5684818114628,1415.5735080386517,1365.4349577363,1333.430901783591,1276.166374742556,1250.7505040186825,1237.2148153820065,1232.9389359691145,1227.8328438730168,1218.516185371996,1185.2920058110433,1174.5800060404586,1172.0696144921542,1126.0950334256443,1098.4033437840824,1089.7031091561817,1088.2444238609328,1074.3003087780903,1051.9865152827506,1045.538404935172,1031.9489812045092,1019.5008345827982,1007.3733174449126,1004.3706705820103,996.346152150619,979.2985872746425,967.3986160126037,964.797357117872,942.2002520007163,934.1737200866155,927.0953746055075,917.402819899194,916.7563591350944,890.4435906432068,882.5731946094143,880.7095926456942,87

In [25]:
    val topConceptTerms = topTermsInTopConcepts(svd, 10, 10, termIds)

topConceptTerms = ArrayBuffer(ArraySeq((nagar,0.3615395072855477), (circle,0.35542560363360254), (kempegowda,0.2570416283692346), (cross,0.23695530967627892), (gate,0.22506285351640026), (bus,0.1834523684953276), (malleshwaram,0.17994799112252052), (palya,0.17350774722971443), (market,0.16611225884788694), (station,0.15574699531556352)), ArraySeq((nagar,3.135937356383428E-5), (kempegowda,2.2559554639783874E-5), (circle,2.199482621303983E-5), (malleshwaram,1.5797751080477013E-5), (palya,1.5232357883936468E-5), (gate,1.444017196184505E-5), (shivajinagar,9.844493304885828E-6), (yeshwantpur,9.478650648317606E-6), (layout,9.032737363925958E-6), (sirsi,8.713706911768308E-6)), ArraySeq((nagar,8.443547495771142E-4), (circle,8.20288911586288E-4), (kempegowda,6.0192449...


ArrayBuffer(ArraySeq((nagar,0.3615395072855477), (circle,0.35542560363360254), (kempegowda,0.2570416283692346), (cross,0.23695530967627892), (gate,0.22506285351640026), (bus,0.1834523684953276), (malleshwaram,0.17994799112252052), (palya,0.17350774722971443), (market,0.16611225884788694), (station,0.15574699531556352)), ArraySeq((nagar,3.135937356383428E-5), (kempegowda,2.2559554639783874E-5), (circle,2.199482621303983E-5), (malleshwaram,1.5797751080477013E-5), (palya,1.5232357883936468E-5), (gate,1.444017196184505E-5), (shivajinagar,9.844493304885828E-6), (yeshwantpur,9.478650648317606E-6), (layout,9.032737363925958E-6), (sirsi,8.713706911768308E-6)), ArraySeq((nagar,8.443547495771142E-4), (circle,8.20288911586288E-4), (kempegowda,6.019244975650276E-4), (malleshwaram,4.2160679738337606E-4), (palya,4.065177120033993E-4), (bus,3.93029872468649E-4), (bgcolor,3.207865845309462E-4), (layout,2.9150135397739384E-4), (hospital,2.7725181986729824E-4), (corporation,2.7251899293350215E-4)), Arra

In [26]:
    val topConceptDocs = topDocsInTopConcepts(svd, 10, 10, docIds)

topConceptDocs = ArrayBuffer(ArraySeq((List of BMTC routes,0.999959129345759), (Mercat cross,0.003192017277100499), (List of hospitals in the Republic of Ireland,0.003056784190751809), (Occupy movement in the United States,0.002128467523640035), (Dartmouth Steam Railway,0.0021086211923507995), (List of terrorist incidents in 2002,0.0019628276865620935), (Attapur,0.0017069713656653657), (History of Knott's Berry Farm,0.0015942864887940557), (List of convicts on the First Fleet,0.0015451302135670935), (List of New York City Subway stations in Brooklyn,0.0012331631865231789)), ArraySeq((List of BMTC routes,3.502801412206937E-5), (Joonas Myyrä,0.0), (Willmar, MN µSA,0.0), (Missio Immaculatae,0.0), (Aethopyga mystacalis,0.0), (Asanuma Inejirō,0.0), (Ayios Vlasios,...


ArrayBuffer(ArraySeq((List of BMTC routes,0.999959129345759), (Mercat cross,0.003192017277100499), (List of hospitals in the Republic of Ireland,0.003056784190751809), (Occupy movement in the United States,0.002128467523640035), (Dartmouth Steam Railway,0.0021086211923507995), (List of terrorist incidents in 2002,0.0019628276865620935), (Attapur,0.0017069713656653657), (History of Knott's Berry Farm,0.0015942864887940557), (List of convicts on the First Fleet,0.0015451302135670935), (List of New York City Subway stations in Brooklyn,0.0012331631865231789)), ArraySeq((List of BMTC routes,3.502801412206937E-5), (Joonas Myyrä,0.0), (Willmar, MN µSA,0.0), (Missio Immaculatae,0.0), (Aethopyga mystacalis,0.0), (Asanuma Inejirō,0.0), (Ayios Vlasios,0.0), (Loren Babcock,0.0), (MTR M-Train EMU,0.0), (Loya Paktya,0.0)), ArraySeq((List of BMTC routes,8.580718062225994E-4), (List of minor planets: 216001–217000,3.613106354945373E-4), (List of hospitals in the Republic of Ireland,3.9551803639559214

In [27]:
    for ((terms, docs) <- topConceptTerms.zip(topConceptDocs)) {
      println("Concept terms: " + terms.map(_._1).mkString(", "))
      println("Concept docs: " + docs.map(_._1).mkString(", "))
      println()
    }

Concept terms: nagar, circle, kempegowda, cross, gate, bus, malleshwaram, palya, market, station
Concept docs: List of BMTC routes, Mercat cross, List of hospitals in the Republic of Ireland, Occupy movement in the United States, Dartmouth Steam Railway, List of terrorist incidents in 2002, Attapur, History of Knott's Berry Farm, List of convicts on the First Fleet, List of New York City Subway stations in Brooklyn

Concept terms: nagar, kempegowda, circle, malleshwaram, palya, gate, shivajinagar, yeshwantpur, layout, sirsi
Concept docs: List of BMTC routes, Joonas Myyrä, Willmar, MN µSA, Missio Immaculatae, Aethopyga mystacalis, Asanuma Inejirō, Ayios Vlasios, Loren Babcock, MTR M-Train EMU, Loya Paktya

Concept terms: nagar, circle, kempegowda, malleshwaram, palya, bus, bgcolor, layout, hospital, corporation
Concept docs: List of BMTC routes, List of minor planets: 216001–217000, List of hospitals in the Republic of Ireland, On-deck, Meanings of minor planet names: 98001–99000, Unive

In [28]:
    val queryEngine = new LSAQueryEngine(svd, termIds, docIds, termIdfs)

queryEngine = com.cloudera.datascience.lsa.LSAQueryEngine@2d4c4c25


com.cloudera.datascience.lsa.LSAQueryEngine@2d4c4c25

In [29]:
    queryEngine.printTopTermsForTerm("algorithm")

(algorithm,0.9999999999999996), (fundamentally,0.9538345102693473), (propagate,0.9505437087708588), (digit,0.9201524010684025), (method,0.8969983021151294), (precision,0.8546089493159822), (zero,0.8501539662034177), (extended,0.8122851267306269), (corresponding,0.8037959481075951), (operand,0.7939031488284806)


In [46]:
    queryEngine.printTopTermsForTerm("kempegowda")

(kempegowda,0.9999999999999998), (sirsi,0.9999999950716311), (talkies,0.9999999950716311), (huskur,0.9999999950716311), (palya,0.999999995071631), (mayohall,0.999999995071631), (ulsoor,0.999999995071631), (puram,0.999999995071631), (hebbal,0.999999995071631), (shivananda,0.999999995071631)


lastException: Throwable = null


In [47]:
    queryEngine.printTopTermsForTerm("malleshwaram")

(sirsi,0.9999999999999999), (talkies,0.9999999999999999), (huskur,0.9999999999999999), (palya,0.9999999999999998), (mayohall,0.9999999999999998), (ulsoor,0.9999999999999998), (puram,0.9999999999999998), (hebbal,0.9999999999999998), (shivananda,0.9999999999999998), (kottigepalya,0.9999999999999998)


In [40]:
    queryEngine.printTopDocsForTerm("fir")

(Nucleotide excision repair,10.710033149931384), (Anion gap,7.84620536785346), (Pinophyta,7.487784309750477), (Elastin-like polypeptides,5.093414920174936), (Castanopsis,3.9750662652860713), (Women in journalism,3.3042198665154268), (History of paper,3.2653481483386155), (Bass–Serre theory,3.259487376006185), (Vireo,2.778234438413666), (Blue cone monochromacy,2.675827851264626)


lastException: Throwable = null


In [41]:
    queryEngine.printTopDocsForTerm("graph")

(Bass–Serre theory,259.02987839396195), (Davis Cup,25.7209623643185), (Richard Brauer,17.914899691135826), (Pinophyta,14.806024638106456), (Bullet graph,10.325770938727308), (2012–13 in Spanish football,9.954872034593345), (Revival Centres International,8.881976921309867), (XDI,8.328890143997462), (2018–19 Danish 2nd Divisions,8.32873112913863), (The Trouble with Physics,8.29508374129109)


In [48]:
    queryEngine.printTopDocsForDoc("BAR domain")

(BAR domain,0.9999999999999998), (Amino acid transporter,0.9205773602633391), (TMC8,0.9119419268739763), (ATP-binding cassette transporter,0.8946194434694467), (C1QTNF5,0.8757624265740518), (Viperin,0.8716276664686162), (Argininosuccinate synthase,0.8658697344826495), (CHCHD2,0.8620463975217801), (SH3BGRL3,0.8573977626641787), (MPDZ,0.8438025466099988)


In [49]:
    queryEngine.printTopDocsForDoc("List of BMTC routes")

(List of BMTC routes,1.0000000000000004), (Cross that bridge,0.9134903449403701), (Internet Corporation for Assigned Names and Numbers,0.8464025003110763), (Raja Nagar,0.7818170958040493), (On-deck,0.6798722913522431), (List of progestogen esters,0.6776934956016234), (Mercat cross,0.673682545526864), (List of hospitals in the Republic of Ireland,0.6653245508488901), (Mania, Dildarnagar,0.6547977141247985), (Rawatpur,0.6421826009698794)


In [50]:
    queryEngine.printTopDocsForDoc("ATP-binding cassette transporter")

(ATP-binding cassette transporter,0.9999999999999993), (Amino acid transporter,0.9892641423840888), (Gold nanoparticles in drug delivery,0.8960418601948735), (BAR domain,0.8946194434694467), (Macromphaliinae,0.8325449758473372), (TMC8,0.8285080560692341), (Peter D. Mitchell,0.7631180065493101), (Argininosuccinate synthase,0.7566217510128299), (Viperin,0.7497435406349588), (C1QTNF5,0.7395854285959019)


In [51]:
    queryEngine.printTopDocsForTermQuery(Seq("nagar", "circle"))

(List of BMTC routes,76521.96234689369), (Mercat cross,232.92649356851106), (List of hospitals in the Republic of Ireland,229.07562958166176), (Apostolic Vicariate of Northern Germany,169.47205248407943), (Attapur,124.27923173497874), (SpellForce: The Order of Dawn,81.28045229856237), (List of terrorist incidents in 2002,72.57781363588069), (History of Knott's Berry Farm,67.40714481963052), (Krishnadevaraya College of Dental Sciences and Hospital,64.72745063670381), (A63 road,57.91104513384207)
